In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Stomach_Cancer"
cohort = "GSE208099"

# Input paths
in_trait_dir = "../../input/GEO/Stomach_Cancer"
in_cohort_dir = "../../input/GEO/Stomach_Cancer/GSE208099"

# Output paths
out_data_file = "../../output/preprocess/Stomach_Cancer/GSE208099.csv"
out_gene_data_file = "../../output/preprocess/Stomach_Cancer/gene_data/GSE208099.csv"
out_clinical_data_file = "../../output/preprocess/Stomach_Cancer/clinical_data/GSE208099.csv"
json_path = "../../output/preprocess/Stomach_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Let's first list the directory contents to understand what files are available
import os

print("Files in the cohort directory:")
files = os.listdir(in_cohort_dir)
print(files)

# Adapt file identification to handle different naming patterns
soft_files = [f for f in files if 'soft' in f.lower() or '.soft' in f.lower() or '_soft' in f.lower()]
matrix_files = [f for f in files if 'matrix' in f.lower() or '.matrix' in f.lower() or '_matrix' in f.lower()]

# If no files with these patterns are found, look for alternative file types
if not soft_files:
    soft_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]
if not matrix_files:
    matrix_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]

print("Identified SOFT files:", soft_files)
print("Identified matrix files:", matrix_files)

# Use the first files found, if any
if len(soft_files) > 0 and len(matrix_files) > 0:
    soft_file = os.path.join(in_cohort_dir, soft_files[0])
    matrix_file = os.path.join(in_cohort_dir, matrix_files[0])
    
    # 2. Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # 3. Obtain the sample characteristics dictionary from the clinical dataframe
    sample_characteristics_dict = get_unique_values_by_row(clinical_data)
    
    # 4. Explicitly print out all the background information and the sample characteristics dictionary
    print("\nBackground Information:")
    print(background_info)
    print("\nSample Characteristics Dictionary:")
    print(sample_characteristics_dict)
else:
    print("No appropriate files found in the directory.")


Files in the cohort directory:
['GSE208099_family.soft.gz', 'GSE208099_series_matrix.txt.gz']
Identified SOFT files: ['GSE208099_family.soft.gz']
Identified matrix files: ['GSE208099_series_matrix.txt.gz']

Background Information:
!Series_title	"Gene expression analysis of M and SM gastric cancer"
!Series_summary	"The objective of this study was to identify genes and pathways involved in submucosal invasion of early gastric cancer through comprehensive gene expression analysis."
!Series_overall_design	"8 cases with intramucosal gastric cancer (M cancer) and 8 cases with early gastric cancer with submucosal invasion ≥ 500 μm (SM cancer) were enrolled in this study. Biopsies were taken from both tumor site and background normal mucosa."

Sample Characteristics Dictionary:
{0: ['gender: M', 'gender: F'], 1: ['tissue: adenocarcinoma', 'tissue: normal mucosa']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
is_gene_available = True  # Based on background information, this dataset contains gene expression data

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
trait_row = 1  # 'tissue' row contains information about whether the sample is cancer or normal
age_row = None  # Age information is not available in the sample characteristics
gender_row = 0  # Gender information is available

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert tissue type to binary trait (1 for cancer, 0 for normal)."""
    if isinstance(value, str) and ":" in value:
        value = value.split(":", 1)[1].strip().lower()
    else:
        value = str(value).lower()
    
    if "adenocarcinoma" in value or "cancer" in value or "tumor" in value:
        return 1
    elif "normal" in value:
        return 0
    return None

def convert_gender(value):
    """Convert gender to binary (0 for female, 1 for male)."""
    if isinstance(value, str) and ":" in value:
        value = value.split(":", 1)[1].strip().upper()
    else:
        value = str(value).upper()
    
    if value == "F" or value == "FEMALE":
        return 0
    elif value == "M" or value == "MALE":
        return 1
    return None

# 3. Save Metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Assuming clinical_data is already available from a previous step
    # If not, it would require proper parsing of the GEO matrix file with appropriate header handling
    
    # Load the sample characteristics dictionary directly
    sample_char_dict = {0: ['gender: M', 'gender: F'], 1: ['tissue: adenocarcinoma', 'tissue: normal mucosa']}
    
    # Create a DataFrame to mimic the structure expected by geo_select_clinical_features
    clinical_data = pd.DataFrame()
    for row_idx, values in sample_char_dict.items():
        clinical_data[row_idx] = values
    
    # Extract and process clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the clinical data
    preview = preview_df(selected_clinical_df)
    print("Clinical Data Preview:")
    print(preview)
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Clinical Data Preview:
{0: [nan, 1.0], 1: [0.0, nan]}
Clinical data saved to ../../output/preprocess/Stomach_Cancer/clinical_data/GSE208099.csv


### Step 3: Gene Data Extraction

In [4]:
# Use the helper function to get the proper file paths
soft_file_path, matrix_file_path = geo_get_relevant_filepaths(in_cohort_dir)

# Extract gene expression data
try:
    gene_data = get_genetic_data(matrix_file_path)
    
    # Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
    
    # Print shape to understand the dataset dimensions
    print(f"\nGene expression data shape: {gene_data.shape}")
    
except Exception as e:
    print(f"Error extracting gene data: {e}")


First 20 gene/probe identifiers:
Index(['A_19_P00315452', 'A_19_P00315492', 'A_19_P00315493', 'A_19_P00315502',
       'A_19_P00315506', 'A_19_P00315518', 'A_19_P00315519', 'A_19_P00315529',
       'A_19_P00315541', 'A_19_P00315543', 'A_19_P00315551', 'A_19_P00315581',
       'A_19_P00315584', 'A_19_P00315593', 'A_19_P00315603', 'A_19_P00315625',
       'A_19_P00315627', 'A_19_P00315631', 'A_19_P00315641', 'A_19_P00315647'],
      dtype='object', name='ID')

Gene expression data shape: (58201, 32)


### Step 4: Gene Identifier Review

In [5]:
# Looking at the gene identifiers, these appear to be Agilent microarray probe IDs,
# not standard human gene symbols. These identifiers (A_19_PXXXXXXXX format) are typical
# of Agilent microarray platforms and need to be mapped to actual gene symbols.

# The format "A_19_P00315452" indicates these are probe IDs from an Agilent microarray platform,
# not standard human gene symbols like "TP53", "EGFR", etc.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
try:
    # Use the correct variable name from previous steps
    gene_annotation = get_gene_annotation(soft_file_path)
    
    # 2. Preview the gene annotation dataframe
    print("Gene annotation preview:")
    print(preview_df(gene_annotation))
    
except UnicodeDecodeError as e:
    print(f"Unicode decoding error: {e}")
    print("Trying alternative approach...")
    
    # Read the file with Latin-1 encoding which is more permissive
    import gzip
    import pandas as pd
    
    # Manually read the file line by line with error handling
    data_lines = []
    with gzip.open(soft_file_path, 'rb') as f:
        for line in f:
            # Skip lines starting with prefixes we want to filter out
            line_str = line.decode('latin-1')
            if not line_str.startswith('^') and not line_str.startswith('!') and not line_str.startswith('#'):
                data_lines.append(line_str)
    
    # Create dataframe from collected lines
    if data_lines:
        gene_data_str = '\n'.join(data_lines)
        gene_annotation = pd.read_csv(pd.io.common.StringIO(gene_data_str), sep='\t', low_memory=False)
        print("Gene annotation preview (alternative method):")
        print(preview_df(gene_annotation))
    else:
        print("No valid gene annotation data found after filtering.")
        gene_annotation = pd.DataFrame()
    
except Exception as e:
    print(f"Error extracting gene annotation data: {e}")
    gene_annotation = pd.DataFrame()


Gene annotation preview:
{'ID': ['GE_BrightCorner', 'DarkCorner', 'A_21_P0014386', 'A_33_P3396872', 'A_33_P3267760'], 'CONTROL_TYPE': ['pos', 'pos', 'FALSE', 'FALSE', 'FALSE'], 'REFSEQ': [nan, nan, nan, 'NM_001105533', nan], 'GB_ACC': [nan, nan, nan, 'NM_001105533', nan], 'LOCUSLINK_ID': [nan, nan, nan, 79974.0, 54880.0], 'GENE_SYMBOL': [nan, nan, nan, 'CPED1', 'BCOR'], 'GENE_NAME': [nan, nan, nan, 'cadherin-like and PC-esterase domain containing 1', 'BCL6 corepressor'], 'UNIGENE_ID': [nan, nan, nan, 'Hs.189652', nan], 'ENSEMBL_ID': [nan, nan, nan, nan, 'ENST00000378463'], 'ACCESSION_STRING': [nan, nan, nan, 'ref|NM_001105533|gb|AK025639|gb|BC030538|tc|THC2601673', 'ens|ENST00000378463'], 'CHROMOSOMAL_LOCATION': [nan, nan, 'unmapped', 'chr7:120901888-120901947', 'chrX:39909128-39909069'], 'CYTOBAND': [nan, nan, nan, 'hs|7q31.31', 'hs|Xp11.4'], 'DESCRIPTION': [nan, nan, nan, 'Homo sapiens cadherin-like and PC-esterase domain containing 1 (CPED1), transcript variant 2, mRNA [NM_001105533

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns to use for mapping based on the gene annotation preview
# Based on the preview, we need to map from 'ID' (probe identifier) to 'GENE_SYMBOL' (gene symbols)
probe_col = 'ID'
gene_col = 'GENE_SYMBOL'

# Print selected columns to confirm our choice
print(f"Using {probe_col} as probe identifier column and {gene_col} as gene symbol column")

# 2. Get a gene mapping dataframe
gene_mapping = get_gene_mapping(gene_annotation, probe_col, gene_col)
print(f"Created gene mapping dataframe with shape: {gene_mapping.shape}")

# Preview the mapping to verify structure
print("Gene mapping preview:")
print(gene_mapping.head())

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
try:
    gene_data = apply_gene_mapping(gene_data, gene_mapping)
    print(f"Converted gene expression data shape: {gene_data.shape}")
    print("First 10 gene symbols after mapping:")
    print(gene_data.index[:10])
    
    # Save the gene expression data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data.to_csv(out_gene_data_file)
    print(f"Gene expression data saved to {out_gene_data_file}")
    
except Exception as e:
    print(f"Error applying gene mapping: {e}")


Using ID as probe identifier column and GENE_SYMBOL as gene symbol column
Created gene mapping dataframe with shape: (48862, 2)
Gene mapping preview:
               ID    Gene
3   A_33_P3396872   CPED1
4   A_33_P3267760    BCOR
5    A_32_P194264   CHAC2
6    A_23_P153745   IFI30
10  A_21_P0014180  GPR146


Converted gene expression data shape: (29222, 32)
First 10 gene symbols after mapping:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A1CF-2', 'A1CF-3', 'A2M', 'A2M-1',
       'A2M-AS1', 'A2ML1', 'A2MP1'],
      dtype='object', name='Gene')


Gene expression data saved to ../../output/preprocess/Stomach_Cancer/gene_data/GSE208099.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the obtained gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print(f"First few normalized gene symbols: {list(normalized_gene_data.index[:10])}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Load the clinical data from the previously saved file
try:
    clinical_data = pd.read_csv(out_clinical_data_file)
    print(f"Loaded clinical data with shape: {clinical_data.shape}")
    print(f"Clinical data columns: {clinical_data.columns.tolist()}")
except Exception as e:
    print(f"Error loading clinical data: {e}")
    # If there's an issue loading the data, attempt to recreate it
    clinical_data = pd.DataFrame()
    if trait_row is not None:
        print("Regenerating clinical data from original sources...")
        # Get clinical data from the matrix file again
        _, clinical_raw = get_background_and_clinical_data(matrix_file_path)
        clinical_data = geo_select_clinical_features(
            clinical_df=clinical_raw,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            gender_row=gender_row,
            convert_gender=convert_gender
        )

# Transpose clinical data to ensure proper format for linking
if not clinical_data.empty:
    clinical_data_transposed = clinical_data.T
    # Rename the index column to ensure proper linking
    if trait in clinical_data_transposed.columns:
        # Link clinical and genetic data
        linked_data = geo_link_clinical_genetic_data(clinical_data_transposed, normalized_gene_data)
        print(f"Linked data shape: {linked_data.shape}")
        print(f"Linked data columns (first few): {linked_data.columns[:10].tolist()}")
        
        # Check if trait column exists in linked data
        if trait in linked_data.columns:
            # 3. Handle missing values in the linked data
            linked_data = handle_missing_values(linked_data, trait)
            print(f"After handling missing values, linked data shape: {linked_data.shape}")
            
            # 4. Check if the trait and demographic features are biased
            is_trait_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
            
            # 5. Validate the data quality and save the validation results
            is_usable = validate_and_save_cohort_info(
                is_final=True, 
                cohort=cohort, 
                info_path=json_path, 
                is_gene_available=True,  
                is_trait_available=True,
                is_biased=is_trait_biased, 
                df=linked_data,
                note="Dataset contains gene expression data comparing adenocarcinoma vs normal mucosa in stomach tissue."
            )
            
            # 6. Save the linked data if it's usable
            print(f"Data quality check result: {'Usable' if is_usable else 'Not usable'}")
            if is_usable:
                os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
                linked_data.to_csv(out_data_file, index=True)
                print(f"Linked data saved to {out_data_file}")
            else:
                print(f"Data quality check failed. The dataset cannot be used for association studies.")
        else:
            print(f"Trait column '{trait}' not found in linked data. Available columns: {linked_data.columns[:5].tolist()}")
            is_usable = validate_and_save_cohort_info(
                is_final=True, 
                cohort=cohort, 
                info_path=json_path, 
                is_gene_available=True,  
                is_trait_available=False,
                is_biased=True, 
                df=pd.DataFrame(),
                note="Clinical data structure incompatible with trait analysis requirements."
            )
            print("Data quality check failed. The trait column was not properly linked.")
    else:
        print(f"Trait column '{trait}' not found in clinical data. Available columns: {clinical_data_transposed.columns.tolist()}")
        is_usable = validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True,  
            is_trait_available=False,
            is_biased=True, 
            df=pd.DataFrame(),
            note="Clinical data lacks the specific trait column needed for analysis."
        )
        print("Data quality check failed. Required trait information is missing.")
else:
    print("No clinical data available for this cohort.")
    is_usable = validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True,  
        is_trait_available=False,
        is_biased=True, 
        df=pd.DataFrame(),
        note="Dataset lacks clinical annotations required for association studies."
    )
    print("Data quality check failed. No clinical data available.")

Normalized gene data shape: (20778, 32)
First few normalized gene symbols: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1', 'A3GALT2', 'A4GALT', 'A4GNT']


Normalized gene data saved to ../../output/preprocess/Stomach_Cancer/gene_data/GSE208099.csv
Loaded clinical data with shape: (2, 2)
Clinical data columns: ['0', '1']
Trait column 'Stomach_Cancer' not found in clinical data. Available columns: [0, 1]
Abnormality detected in the cohort: GSE208099. Preprocessing failed.
Data quality check failed. Required trait information is missing.
